# Les requêtes de modification

* Auteurs : Rémi Leduc, Léo Rouger, Clément Caillard

Ce chapitre traite des différentes requêtes de modification ([insertion](sec:insertion), [modification](sec:modification) et [suppression](sec:suppression)).

(sec:insertion)=
## Insertion

Pour la création d'une nouvelle collection, il est nécessaire de sélectionner une base. Si elle n'existe pas, elle sera alors créée.

In [1]:
use voitures

switched to db voitures

### Création d'une collection 

Il est nécessaire d'utiliser la commande `db.createCollection("NomDeLaNouvelleCollection")` pour créer une nouvelle collection au sein de la base sélectionnée.  
Exemple:

In [2]:
db.createCollection("ventes")

{ "ok" : 1 }

Ici, on crée par exemple une collection "ventes" au sein de la base "voitures".  
:warning: **Si vous tentez d'exécuter plusieurs fois la commande, une erreur surviendra car votre collection existe déjà.** 

### Ajout de nouveaux documents 

La commande `db.NomDeLaCollection.insert([])` permet l'ajout d'une liste de document au sein de la collection.  
Exemple:

In [3]:
db.ventes.insert([{"nom" : "C1"}, {"nom" : "C2"}])

BulkWriteResult({
	"writeErrors" : [ ],
	"writeConcernErrors" : [ ],
	"nInserted" : 2,
	"nUpserted" : 0,
	"nMatched" : 0,
	"nModified" : 0,
	"nRemoved" : 0,
	"upserted" : [ ]
})

Si on exécute la ligne suivante, on retrouve bien les documents créés :

In [4]:
db.ventes.find({})

{ "_id" : ObjectId("6083c40061c1eaaa4b9b59ce"), "nom" : "C1" }
{ "_id" : ObjectId("6083c40061c1eaaa4b9b59cf"), "nom" : "C2" }

(sec:modification)=
## Modification
### Remplacement d'un document

In [5]:
db.ventes.update(
	{"nom": "C1"},
	{"nom": "C1", "marque": "Citroën"}
)

WriteResult({ "nMatched" : 1, "nUpserted" : 0, "nModified" : 1 })

Le document sélectionné sur la première ligne est supprimé et remplacé selon les champs renseignés sur la seconde ligne. Les champs qui ne sont pas renseignés sont donc supprimés.  
:warning: **Seul le premier document de la liste sera modifié.**

### Modification d'un document
Si l'on souhaite conserver les autres champs, il suffit d'inclure la seconde ligne dans un `$set`.

In [6]:
db.ventes.update(
	{"nom": "C2"},
	{$set:
		{"marque": "Citroën"}
	}
)

WriteResult({ "nMatched" : 1, "nUpserted" : 0, "nModified" : 1 })

Ici également, seul le premier document de la liste répondant aux critères de la première ligne sera modifié.

### Modification de plusieurs documents
Pour modifier plusieurs documents à la fois, il est nécessaire d'ajouter `{multi: true}` en fin de requête.

In [7]:
db.ventes.update(
	{"nom" : {$in: ["C1", "C2"]}},
	{$set: {"pays": "France"}},
	{multi: true}
)

WriteResult({ "nMatched" : 2, "nUpserted" : 0, "nModified" : 2 })

Cette requête, par exemple, ajoute un attribut "pays" ayant la valeur "France" aux modèles C1 **et** C2.

### Upsert
L'option `upsert` (mélange de "update" et "insert") permet de mettre une condition sur la requête : si aucun document ne correspond aux conditions indiquées en première ligne, alors un nouveau document est créé par les champs renseignés sur la seconde ligne.

In [8]:
db.ventes.update(
	{"nom": "C1"},
	{$set : {"nom": "C1", "Nombre de roues": 4}},
	{upsert: true}
)

WriteResult({ "nMatched" : 1, "nUpserted" : 0, "nModified" : 1 })

Ici, on ajoute une nouvelle variable "Nombre de roues" à laquelle on attribue la valeur 4 au modèle "C1".

In [9]:
db.ventes.update(
	{"nom": "Twingo"},
	{$set : {"Nombre de roues": 4}},
	{upsert: true}
)

WriteResult({
	"nMatched" : 0,
	"nUpserted" : 1,
	"nModified" : 0,
	"_id" : ObjectId("6083c4019dd738d6e28cb8b6")
})

Cette fois-ci, un nouveau document est ajouté à la base.

(sec:suppression)=
## Suppression

Comme toutes les bonnes choses ont une fin, il est possible de supprimer une base, ou bien, sans être aussi radical, des éléments plus précis.
Nous allons voir ici comment procéder aux différentes suppressions :

### Suppression d'une base entière
```js
db.dropDatabase()
```
Lors de l'exécution de cette commande, c'est la base **courante**, celle qui est pointée par _db_, qui sera supprimée.
```{admonition} Important !
:class: warning
Notez bien que tous les index qui pouvaient exister dans les collections de cette base seront également supprimés !
```

### Suppression d'une collection
```js
db.nomDeLaCollection.drop()
```
Ici encore, les index éventuellement présents dans la collection supprimée seront eux aussi effacés.

### Suppression de documents dans une collection
Nous voilà arrivés au gros morceau...
Lorsque l'on veut supprimer certains documents en particulier **sans toucher aux index**, il faut utiliser la commande suivante :
```js
db.nomDeLaCollection.remove({})
```
Lorsque l'on passe en argument un document vide, comme dans l'exemple ci-dessus, on supprime toutes les données contenues dans la collection, mais on en conserve la structure, donc les index.

La fonction _remove_ peut également recevoir des documents précis en argument :
* Condition sous la forme d'un document masque :  
   Tous les documents correspondants à la sélection seront supprimés, par exemple tous ceux dont l'attribut "marque" correspond à "Citroën" :

In [10]:
db.ventes.remove({"marque" : "Citroën"})

WriteResult({ "nRemoved" : 2 })

* Suppression d'un seul document :  
   Pour ce faire, il convient d'utiliser l'attribut "_id" puisqu'il est unique :
```js
db.nomDeLaCollection.remove({"_id" : ObjectId("5612c6c0a5c56580cfacc342")})
``` 

Et voilà, vous savez tout sur la création, la suppression et la modification de bases !